In [1]:
import importlib
import strategies.ema_crossover_v1.strategy as ema_crossover_v1

importlib.reload(ema_crossover_v1)
EMACrossoverRSIStrategy = ema_crossover_v1.EMACrossoverRSIStrategy

## Constants

In [2]:
from datetime import datetime, timezone

RISK_FREE_RATE = 0.027 # Example risk-free rate, adjust as needed
TEST_SET_START_DATE = datetime(2021, 1, 1, tzinfo=timezone.utc)
TEST_SET_END_DATE = datetime(2023, 12, 31, tzinfo=timezone.utc)
EVALUATION_START_DATE = datetime(2023, 1, 1, tzinfo=timezone.utc)
EVALUATION_END_DATE = datetime(2025, 6, 30, tzinfo=timezone.utc)
MARKET = "BITVAVO"
TEST_SYMBOLS = ["BTC/EUR", "ETH/EUR", "LTC/EUR", "XRP/EUR", "ADA/EUR"]
EVALUATION_SYMBOLS = [
    "SOL/EUR", "DOT/EUR", "LINK/EUR", "MATIC/EUR", "AVAX/EUR"
]
storage_path = "./data"

## Best performing params

In [3]:
variant_one = {
    'ema_short_period': 75,
    'ema_long_period': 100,
    'rsi_period': 14,
    'rsi_overbought_threshold': 70,
    'rsi_oversold_threshold': 30,
    'alignment_window_size': 12,
    'ema_time_frame': '1d',
    'rsi_time_frame': '4h'
}
variant_two = {
    'ema_short_period': 75,
    'ema_long_period': 100,
    'rsi_period': 14,
    'rsi_overbought_threshold': 70,
    'rsi_oversold_threshold': 30,
    'alignment_window_size': 24,
    'ema_time_frame': '1d',
    'rsi_time_frame': '4h'
}
variant_three = {
    'ema_short_period': 50,
    'ema_long_period': 100,
    'rsi_period': 14,
    'rsi_overbought_threshold': 70,
    'rsi_oversold_threshold': 30,
    'alignment_window_size': 24,
    'ema_time_frame': '1d',
    'rsi_time_frame': '4h'
}
variant_four = {
    'ema_short_period': 50,
    'ema_long_period': 100,
    'rsi_period': 14,
    'rsi_overbought_threshold': 70,
    'rsi_oversold_threshold': 30,
    'alignment_window_size': 12,
    'ema_time_frame': '4h',
    'rsi_time_frame': '1h'
}
variant_five = {
    'ema_short_period': 75,
    'ema_long_period': 100,
    'rsi_period': 14,
    'rsi_overbought_threshold': 70,
    'rsi_oversold_threshold': 30,
    'alignment_window_size': 24,
    'ema_time_frame': '1d',
    'rsi_time_frame': '1h'
}

variants = [
    variant_one,
    variant_two,
    variant_three,
    variant_four,
    variant_five
]

## Data Preparation

In [4]:
from investing_algorithm_framework import select_backtest_date_ranges, \
    download, SnapshotInterval

test_data = download(
    symbol="BTC/EUR",
    start_date=TEST_SET_START_DATE,
    end_date=TEST_SET_END_DATE,
    market=MARKET,
    time_frame="1d",
    save=True,
    storage_path=storage_path
)
evaluation_data = download(
    symbol="BTC/EUR",
    start_date=EVALUATION_START_DATE,
    end_date=EVALUATION_END_DATE,
    market=MARKET,
    time_frame="1d",
    save=True,
    storage_path=storage_path
)
test_backtest_date_ranges = select_backtest_date_ranges(
    df=test_data, window=730
)
evaluation_backtest_date_ranges = select_backtest_date_ranges(
    df=evaluation_data, window=730
)

for date_range in test_backtest_date_ranges:
    print(
        f"Backtest Date Range ({date_range.name}): " +
        f"{date_range.start_date} to {date_range.end_date}"
    )

for date_range in evaluation_backtest_date_ranges:
    print(
        f"Evaluation Date Range ({date_range.name}): " +
        f"{date_range.start_date} to {date_range.end_date}"
    )

Backtest Date Range (UpTurn): 2021-07-20 00:00:00+00:00 to 2023-07-20 00:00:00+00:00
Backtest Date Range (DownTurn): 2021-03-11 00:00:00+00:00 to 2023-03-11 00:00:00+00:00
Backtest Date Range (SideWays): 2021-12-31 00:00:00+00:00 to 2023-12-31 00:00:00+00:00
Evaluation Date Range (UpTurn): 2023-01-07 00:00:00+00:00 to 2025-01-06 00:00:00+00:00
Evaluation Date Range (DownTurn): 2023-04-11 00:00:00+00:00 to 2025-04-10 00:00:00+00:00
Evaluation Date Range (SideWays): 2023-01-01 00:00:00+00:00 to 2024-12-31 00:00:00+00:00


## Run event-based backtests

In [ ]:
from tqdm.notebook import tqdm
from investing_algorithm_framework import create_app, DataSource, \
    PortfolioConfiguration, RESOURCE_DIRECTORY


app = create_app(
    config={
        RESOURCE_DIRECTORY: "./resources",
    }
)
app.add_portfolio_configuration(
    PortfolioConfiguration(
        initial_balance=1000,
        trading_symbol="EUR",
        market=MARKET,
    )
)

for variant in tqdm(variants, desc=f"Running backtests for {len(variants)} variants"):
    ema_time_frame = variant["ema_time_frame"]
    rsi_time_frame = variant["rsi_time_frame"]
    ema_data_sources = [
        DataSource(
            symbol=symbol,
            time_frame=ema_time_frame,
            data_type="OHLCV",
            market=MARKET,
            pandas=True,
            window_size=200
        ) for symbol in TEST_SYMBOLS
    ]
    rsi_data_sources = [
        DataSource(
            symbol=symbol,
            time_frame=rsi_time_frame,
            data_type="OHLCV",
            market=MARKET,
            pandas=True,
            window_size=200,
        ) for symbol in TEST_SYMBOLS
    ]
    data_sources = ema_data_sources + rsi_data_sources
    strategy = EMACrossoverRSIStrategy(
        time_unit="hour",
        interval=2,
        symbols=["BTC/EUR"],
        data_sources=data_sources,
        market=MARKET,
        ema_short_period=variant["ema_short_period"],
        ema_long_period=variant["ema_long_period"],
        rsi_period=variant["rsi_period"],
        rsi_oversold_threshold=variant["rsi_oversold_threshold"],
        rsi_overbought_threshold=variant["rsi_overbought_threshold"],
        alignment_window_size=variant["alignment_window_size"],
        ema_time_frame=variant["ema_time_frame"],
        rsi_time_frame=variant["rsi_time_frame"],
    )
    backtests = app.run_backtests(
        strategy=strategy,
        backtest_date_ranges=test_backtest_date_ranges,
        initial_amount=1000,
        snapshot_interval=SnapshotInterval.DAILY,
        risk_free_rate=RISK_FREE_RATE,
        save=False
    )
    for backtest in backtests:
        print(
            f"Backtest for {variant['ema_time_frame']} "
        )
        print(f"  Final value: {backtest.backtest_metrics.final_value:.2f}")
        print(f"  Total Return: {backtest.backtest_metrics.total_return:.2f} ({backtest.backtest_metrics.total_return_percentage:.2%})")
        print(f"  Sharpe Ratio: {backtest.backtest_metrics.sharpe_ratio:.2f}")
        print(f"  Sortino Ratio: {backtest.backtest_metrics.sortino_ratio:.2f}")
        print(f"  Max Drawdown: {backtest.backtest_metrics.max_drawdown:.2%}")
        print(f"  Win Rate: {backtest.backtest_metrics.win_rate}%")
        print(f"  Profit Factor: {backtest.backtest_metrics.profit_factor:.2f}")

Running backtests for 5 variants:   0%|          | 0/5 [00:00<?, ?it/s]

Preparing backtest data:   0%|          | 0/10 [00:00<?, ?it/s]

Running backtest:   0%|          | 0/8761 [00:00<?, ?it/s]

Preparing backtest data:   0%|          | 0/10 [00:00<?, ?it/s]

Running backtest:   0%|          | 0/8761 [00:00<?, ?it/s]

...

Backtest for 1d 
Final Balance: 1124.36 EUR
Total Return: 12436.41%
Sharpe Ratio: 0.32
Max Drawdown: 14.32%

Backtest for 1d 
Final Balance: 1124.36 EUR
Total Return: 12436.41%
Sharpe Ratio: 0.32
Max Drawdown: 14.32%

Backtest for 1d 
Final Balance: 1000.00 EUR
Total Return: 0.00%
Sharpe Ratio: nan
Max Drawdown: 0.00%



Preparing backtest data:   0%|          | 0/10 [00:00<?, ?it/s]

Running backtest:   0%|          | 0/8761 [00:00<?, ?it/s]

Preparing backtest data:   0%|          | 0/10 [00:00<?, ?it/s]

Running backtest:   0%|          | 0/8761 [00:00<?, ?it/s]